<a href="https://colab.research.google.com/github/nreichen/eecs448-CollegeSubredditsCommentGeneration/blob/main/custom_trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os 
GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE = "448 Group: Jedi Order/models/UIUC"
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(GOOGLE_DRIVE_PATH)

Mounted at /content/drive
['checkpoints', 'train_post_set.pickle', 'val_post_set.pickle', 'val_com_set.pickle', 'train_com_set.pickle', 'test_post_set.pickle', 'test_com_set.pickle', 'uiucgenerationmanual.ipynb']
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import locale
print(locale.getpreferredencoding())

def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

UTF-8


In [ ]:
!pip install transformers
!pip install convokit
!pip install datasets
!pip install wandb
!pip install rouge-score
!pip install pandas
!pip install sentencepiece
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pickle
import pandas as pd
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling, EarlyStoppingCallback
from datasets import load_dataset
from convokit import Corpus, download
from torch.utils.data import DataLoader, IterableDataset, Dataset
from torch import optim
import numpy as np
import wandb
import re
from rouge_score import rouge_scorer
from tqdm import tqdm
import nltk
from nltk import word_tokenize
from sklearn.model_selection import train_test_split

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Load and Prepare Dataset


The below block creates a corpus object that contains the data for our subreddit. In this notebook, the subreddit is the UIUC subreddit.

In [ ]:
GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE_DATA = "448 Group: Jedi Order/data/CornellSubreddits/UIUC.corpus"
DATA_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE_DATA)
corpus = Corpus(filename=DATA_PATH)
corpus.print_summary_stats()

Number of Speakers: 35751
Number of Utterances: 657989
Number of Conversations: 66177


Here we obtain the Pandas dataframes from the Corpus

In [ ]:
convo_df = corpus.get_conversations_dataframe()
utt_df = corpus.get_utterances_dataframe()
utt_ids = corpus.get_utterance_ids()

We first sort our dataframe in descending order based on score, from which we pull the posts and comments separately. This is done through checking if the "reply_to" field has an entry or not. If there is nothing the "reply_to" field, then that means the utterance in question is not replying to anything (i.e. a post). Otherwise it is a comment. We then find the top 20,000 posts from the UIUC subreddit.

In [ ]:
temp = utt_df.sort_values("meta.score", ascending=False)
posts = temp[temp['reply_to'].isnull()]
comments = temp[temp['reply_to'].notnull()]
posts = posts[(posts["text"] != '') & (posts["text"] != '[deleted]')  & (posts["text"] != '[removed]') & (posts["text"] != ' ')]
comments = comments[(comments["text"] != '') & (comments["text"] != '[deleted]')  & (comments["text"] != '[removed]') & (comments["text"] != ' ')]
top_posts = posts.head(20000) # specific to UIUC

conversation_ids = list(top_posts["conversation_id"])
#comments.head()

We then create our tokenizer. It will be the GPT2Tokenizer. We split our data as a 75:25:25 split. Subsequently, from each of these id splits, we list out all of the post ids, and find all associated comments.


In [ ]:
train_dataset_to_use = []
val_dataset_to_use = []
test_dataset_to_use = []
train_com = []
val_com = []
test_com = []
train_len = []
val_len = []
test_len = []
train_com_len = []
val_com_len = []
test_com_len = []

train_ids, test_ids = train_test_split(conversation_ids, test_size=0.25, random_state=42)
val_ids, test_ids = train_test_split(test_ids, test_size=0.5, random_state=42)
ids_total = [train_ids, val_ids, test_ids]
post_len = [train_len, val_len, test_len]
com_len = [train_com_len, val_com_len, test_com_len]
posts_datasets_to_use = [train_dataset_to_use, val_dataset_to_use, test_dataset_to_use]
comments_datasets_to_use = [train_com, val_com, test_com]


for index, id_list in enumerate(ids_total):
  for id in id_list:
    dataframe = comments[comments["conversation_id"] == id]
    post = top_posts.loc[id]['text']
    post = re.sub("\n", " ", post)
    post = re.sub("&gt;", "", post)
    post = re.sub("  ", " ", post)
    post = re.sub("&amp;", "", post)
    post = re.sub("#x200B;", "", post)
    post = re.sub(r'http\S+', '[url link]', post)
    post = post + "the top reply was: "
    post_tokens = word_tokenize(post)
    post_len[index].append(len(post_tokens))
    output = dataframe.head(5)
    commented_here = "the post was: " + post
    com_ids = list(output.index)
    running_sum = 0 
    for id_c in com_ids:
      out_text = output.loc[id_c]['text']
      out_text = str(out_text).strip()
      out_text = re.sub(r"\s", " ", out_text)
      out_text = re.sub(r"\n", " ", out_text)
      out_text = re.sub("&gt;", "", out_text)
      out_text = re.sub("  ", " ", out_text)
      out_text = re.sub("&amp;", "", out_text)
      out_text = re.sub("#x200B;", "", out_text)
      out_text = re.sub(r'http\S+', '[url link]', out_text)
      out_text_tokens = word_tokenize(out_text) 
      running_sum += len(out_text_tokens)
      if running_sum > 1024: 
        break 
      commented_here = commented_here + " " + out_text
    comment_tokens = word_tokenize(commented_here)
    com_len[index].append(len(comment_tokens))
    post_utter = "<bos> " + post + " <eos> "
    comment_utter = "<bos>" + commented_here + " <eos> "
    posts_datasets_to_use[index].append(post_utter)
    comments_datasets_to_use[index].append(comment_utter)

In [ ]:
print(posts_datasets_to_use[0][0])
print(post_len[0][0])
#comments_datasets_to_use[0][0]

<bos> I rented a U-Haul truck but I don't have a driver's license. Is there a website where I can rent a driver or something? <eos> 
28


In [ ]:
averages_post = []
boundary_post = []
averages_com = []
boundary_com = []

for datasets in post_len:
  temp = np.array(datasets)
  averages_post.append(np.mean(temp))
  boundary_post.append(np.percentile(temp, 90)) # changed this from 98
for datasets in com_len:
  temp = np.array(datasets)
  averages_com.append(np.mean(temp))
  boundary_com.append(np.percentile(temp, 90)) # changed this from 98
print(averages_post)
print(boundary_post)
print(averages_com)
print(boundary_com)

[89.1362, 88.696, 89.4596]
[182.0, 183.0999999999999, 178.0999999999999]
[181.37573333333333, 181.366, 181.5168]
[403.0, 404.0, 398.0]


In [ ]:
averages_post = []
boundary_post = []
averages_com = []
boundary_com = []
train_post_remove = []
val_post_remove = []
test_post_remove = []
posts_removed_ind = [train_post_remove, val_post_remove, test_post_remove]
train_com_remove = []
val_com_remove = []
test_com_remove = []
coms_removed_ind = [train_com_remove, val_com_remove, test_com_remove]

for datasets in post_len:
  temp = np.array(datasets)
  averages_post.append(np.mean(temp))
  boundary_post.append(np.percentile(temp, 90)) # changed this from 98
for datasets in com_len:
  temp = np.array(datasets)
  averages_com.append(np.mean(temp))
  boundary_com.append(np.percentile(temp, 90)) # changed this from 98
for index1, datasets in enumerate(post_len):
  for index, length in enumerate(datasets):
    if length > boundary_post[index1]:
      posts_removed_ind[index1].append(index)
for index1, datasets in enumerate(com_len):
  for index, length in enumerate(datasets):
    if length > boundary_com[index1]:
      coms_removed_ind[index1].append(index)
for index in range(3):
  temp_pairing = []
  temp_pairing.append(posts_removed_ind[index])
  temp_pairing.append(coms_removed_ind[index])
  our_set = set().union(*temp_pairing)
  for i in sorted(our_set, reverse=True):
    del post_len[index][i]
    del com_len[index][i]
    del posts_datasets_to_use[index][i]
    del comments_datasets_to_use[index][i]
print(len(posts_datasets_to_use[0]))
print(len(comments_datasets_to_use[0]))
print(len(post_len[0]))
#print(coms_removed_ind)
#print(boundary_post)
#print(boundary_com)
#print(post_len[0][0])
#print(com_len[0][0])
#print(len(tokenizer))

12371
12371
12371


In [ ]:
#print(posts_datasets_to_use[0][0])
#print(posts_datasets_to_use[0][0])
#print(len(comments_datasets_to_use[0]))
train_post_path = os.path.join(GOOGLE_DRIVE_PATH, "train_post_set.pickle")
val_post_path = os.path.join(GOOGLE_DRIVE_PATH, "val_post_set.pickle")
test_post_path = os.path.join(GOOGLE_DRIVE_PATH, "test_post_set.pickle")
train_com_path = os.path.join(GOOGLE_DRIVE_PATH, "train_com_set.pickle")
val_com_path = os.path.join(GOOGLE_DRIVE_PATH, "val_com_set.pickle")
test_com_path = os.path.join(GOOGLE_DRIVE_PATH, "test_com_set.pickle")
print(train_post_path)
pickle.dump(posts_datasets_to_use[0], open(train_post_path, "wb"))
pickle.dump(posts_datasets_to_use[1], open(val_post_path, "wb"))
pickle.dump(posts_datasets_to_use[2], open(test_post_path, "wb"))
pickle.dump(comments_datasets_to_use[0], open(train_com_path, "wb"))
pickle.dump(comments_datasets_to_use[1], open(val_com_path, "wb"))
pickle.dump(comments_datasets_to_use[2], open(test_com_path, "wb"))

drive/My Drive/448 Group: Jedi Order/models/UIUC/train_post_set.pickle


In [ ]:
print(len(posts_datasets_to_use[0]))
print(posts_datasets_to_use[0][0])

12371
<bos> I rented a U-Haul truck but I don't have a driver's license. Is there a website where I can rent a driver or something? <eos> 


In [ ]:
posts_datasets_to_use[0] = []
print(len(posts_datasets_to_use[0]))
# print(posts_datasets_to_use[0][0])

0


In [ ]:
train_post_path = os.path.join(GOOGLE_DRIVE_PATH, "train_post_set.pickle")
val_post_path = os.path.join(GOOGLE_DRIVE_PATH, "val_post_set.pickle")
test_post_path = os.path.join(GOOGLE_DRIVE_PATH, "test_post_set.pickle")
train_com_path = os.path.join(GOOGLE_DRIVE_PATH, "train_com_set.pickle")
val_com_path = os.path.join(GOOGLE_DRIVE_PATH, "val_com_set.pickle")
test_com_path = os.path.join(GOOGLE_DRIVE_PATH, "test_com_set.pickle")
posts_datasets_to_use = [[], [], []]
comments_datasets_to_use = [[], [], []]

In [ ]:
posts_datasets_to_use[0] = pickle.load(open(train_post_path, "rb"))
posts_datasets_to_use[1] = pickle.load(open(val_post_path, "rb"))
posts_datasets_to_use[2] = pickle.load(open(test_post_path, "rb"))
comments_datasets_to_use[0] = pickle.load(open(train_com_path, "rb"))
comments_datasets_to_use[1] = pickle.load(open(val_com_path, "rb"))
comments_datasets_to_use[2] = pickle.load(open(test_com_path, "rb"))

In [ ]:
posts_datasets_to_use[0][0]

"<bos> I rented a U-Haul truck but I don't have a driver's license. Is there a website where I can rent a driver or something? <eos> "

In [ ]:
class RedditDataset(Dataset):
  """Dataset for our Reddit thing"""

  def __init__(self, posts, comments, tokenizer, input_length, output_length, dataset_type: str = 'Train'):
    # super().__init__()
    self.posts = posts
    self.comments = comments
    self.dataset_type = dataset_type
    self.input_length = input_length
    self.output_length = output_length
    self.tokenizer = tokenizer

  def __len__(self):
    assert(len(self.posts) == len(self.comments))
    return len(self.posts)

  #def get_labels(self) -> pd.Series:
  #  return self.comments

  def __getitem__(self, idx):
    # check whether it wants a batch encoding or not
    # can you even do it with too long a comment?
    post_text = self.posts[idx]
    comment_text = self.comments[idx]
    dct_input = self.tokenizer(post_text, max_length=self.input_length, truncation=True,
                              padding="max_length", return_tensors="pt")
    dct_output = self.tokenizer(comment_text, max_length=self.output_length, truncation=True,
                               padding="max_length", return_tensors="pt")
    #print(dct_input.input_ids)
    #print(dct_output.input_ids)
    if self.dataset_type == 'Train':
      return_dict = {
        'input_ids': dct_input.input_ids,
        'attention_mask': dct_input.attention_mask,
        'labels': dct_output.input_ids
      }
    else:
      return_dict = {
        'input_ids': dct_input.input_ids,
        'target_text': comment_text,
        'input_text': post_text
      }
    return return_dict

In [ ]:
def train(train_set, train_comments, model, tokenizer, save_every_x_iterations, epochs, batch_size, input_length, output_length,
          lr: float = 1e-5, log: int = 20, wand: bool = False, val_verbose: bool = False, save_name: str = "train", val_set: list = None,
          val_com: list = None):
  """ Prayers """
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model.to(device)

  print("Creating datasets")
  train_data = RedditDataset(train_set, comments=train_comments, tokenizer=tokenizer, input_length=input_length, output_length=output_length)
  trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

  if val_set is not None:
    val_data = RedditDataset(val_set, comments=val_com, tokenizer=tokenizer, input_length=input_length, output_length=output_length)
    valloader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True)
  
  optimizer = optim.Adam(model.parameters(), lr)
  
  print("Beginning Training")
  for e in range(epochs):
    print(f"Training epoch {e + 1}")
    train_loss = []
    temp_loss = []
    for ind, val in tqdm(enumerate(trainloader)):
      input_ids = val["input_ids"].to(device).squeeze(1)
      attention_mask = val["attention_mask"].to(device).squeeze(1)
      labels = val["labels"].to(device).squeeze(1)
      optimizer.zero_grad()
      loss = model.forward(input_ids=input_ids, attention_mask=attention_mask, labels=labels).loss
      loss.backward()
      optimizer.step()
      train_loss.append(loss.item())
      temp_loss.append(loss.item())

      if ind % save_every_x_iterations == 0 and ind != 0:
        print(f"Saving model epoch {e+1} iteration {ind}")
        torch.save(model.cpu().state_dict(), save_name + f'_epoch_{e+1}_iteration_{ind}.bin')
        model.to(device)

      if ind % log == 0:
        if wand:
          wandb.log({'Train loss': np.mean(temp_loss)})
        print(f'Train loss: {np.mean(temp_loss)}')
        temp_loss = []
    print(f'The average loss for this epoch was {np.mean(train_loss)}')
  print('Finished Training!')
  return model

In [ ]:
def get_comments(model, eval_set, eval_com, tokenizer, batch_size, input_length, output_length):
  """yoink"""
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model.to(device).eval()

  print('Creating datasets')
  eval_data = RedditDataset(eval_set, eval_com, tokenizer=tokenizer, input_length=input_length, output_length=output_length, dataset_type="Test")
  evalloader = torch.utils.data.DataLoader(eval_data, batch_size=batch_size, shuffle=True)

  pred_comments, true_comments, input_convo = [], [], []
  print("Running predictions")
  with torch.no_grad():
    for i, val in tqdm(enumerate(evalloader)):
      input_ids = val["input_ids"].to(device).squeeze(1)
      comments_ids = model.generate(input_ids, num_beams=10,
                                    max_length=output_length, early_stopping=True, pad_token_id=tokenizer.eos_token_id, 
                                    do_sample=True, top_k=50, temperature=0.7) #check this
      pred_comments += tokenizer.batch_decode(comments_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
      true_comments += list(val['target_text'])
      input_convo += list(val['input_text'])

  comment_df = pd.DataFrame({"input_text": input_convo, 
                             "target_text": true_comments, "predicted_text": pred_comments})
  return comment_df

def evaluate(model, eval_set, tokenizer, batch_size, input_length, output_length, eval_com,
             rouge_scores: list = ['rouge1', 'rouge2', 'rougeL']):
  scorer = rouge_scorer.RougeScorer(rouge_scores, use_stemmer=False)
  # scores = np.zeros((len(rouge_scores), eval_set.shape[0], 2))
  scores = np.zeros((len(rouge_scores), len(eval_set), 2))

  comments_generated = get_comments(model=model, eval_set=eval_set, eval_com=eval_com, tokenizer=tokenizer, batch_size=batch_size, 
                                    input_length=input_length, output_length=output_length)
  
  print("Scoring predictions")
  for ind, comme in comments_generated.iterrows():
    score = scorer.score(comme["target_text"], comme["predicted_text"])

    for j, key in enumerate(rouge_scores):
      scores[j, ind, :] = score[key].precision, score[key].recall

  rouge_dict = dict()
  means = scores.mean(axis=1)
  for i, key in enumerate(rouge_scores):
    p, r = means[i]
    rouge_dict[key] = {'p': p, 'r': r, 'f': 2*((p*r)/(p+r))}
  print("Evaluation complete!")
  return comments_generated, rouge_dict

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2", bos_token = "<bos>", eos_token ="<eos>")
#model = GPT2LMHeadModel.from_pretrained("gpt2")
model1 = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model1.resize_token_embeddings(len(tokenizer))
#model.resize_token_embeddings(len(tokenizer))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Embedding(50260, 768)

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
gpt2_model = train(posts_datasets_to_use[0], comments_datasets_to_use[0], model, tokenizer, epochs=2, input_length=512, save_every_x_iterations=2000,
                   batch_size=5, output_length=512)

Creating datasets
Beginning Training
Training epoch 1


1it [00:00,  1.27it/s]

Train loss: 3.175663709640503


8it [00:05,  1.36it/s]


KeyboardInterrupt: ignored

In [ ]:
import torch
torch.cuda.empty_cache()

RuntimeError: ignored

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inputs = tokenizer(test_df.input_text.values[0], max_length=512, truncation=True, padding='max_length', return_tensors='pt')
summary_ids = Bart.generate(inputs['input_ids'].to(device), max_length=128, early_stopping=True)
print([Bart_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids])

In [ ]:
posts_datasets_to_use[2][0]

"<bos> ^ the only one I went to is ikenberry, which is nice and has a vast food selection also with enough vegan/gluten free choices, only problem is cost (during school year, maybe? in summer they didn't change for any specific items) and long lines. are any of the other dining halls better/comparable and do they serve special diet foods? <eos> "

In [ ]:
for index, sentence in enumerate(posts_datasets_to_use[2]):
  sentence = sentence[:len(sentence) - 6]
  sentence = sentence + "The top reply was: <eos>"
  posts_datasets_to_use[2][index] = sentence
posts_datasets_to_use[2][0]

"<bos> ^ the only one I went to is ikenberry, which is nice and has a vast food selection also with enough vegan/gluten free choices, only problem is cost (during school year, maybe? in summer they didn't change for any specific items) and long lines. are any of the other dining halls better/comparable and do they serve special diet foods? The top reply was: <eos>"

In [ ]:
comments_datasets_to_use[2]

["<bos> ^ the only one I went to is ikenberry, which is nice and has a vast food selection also with enough vegan/gluten free choices, only problem is cost (during school year, maybe? in summer they didn't change for any specific items) and long lines. are any of the other dining halls better/comparable and do they serve special diet foods? The top reply was: All the dining halls are all you can eat, the A La Carte locations (listed above by /u/palmtree5) are pay per item. No ripoffs, just sometimes meh food. Download the UI Dining app to see the menus for each meal on a daily basis! So how is the only problem cost? It's all the same.... If you're going into a dining hall for a meal, it's all you can eat. There's no pricing of individual items there. If you're going to one of [these]([url link] that's where the items are priced individually. LAR has a lot of vegan options as well. You can look through the menus of all the dining halls [here]([url link] Be sure to change the date to one

In [ ]:
sentence = posts_datasets_to_use[2][0]
sentence = sentence[:len(sentence) - 5]
print(sentence)

<bos> ^ the only one I went to is ikenberry, which is nice and has a vast food selection also with enough vegan/gluten free choices, only problem is cost (during school year, maybe? in summer they didn't change for any specific items) and long lines. are any of the other dining halls better/comparable and do they serve special diet foods? The top reply was: 


In [ ]:
for index, sentence in enumerate(comments_datasets_to_use[2]):
  # should it be: the post was: posts_datasets_to_use[index][index1](including the top reply was:) and then the comments?
  # or should it be: posts_datasets_to_use[index][index1](incliuding the top reply was:) and then the comments
  temp = posts_datasets_to_use[2][index]
  temp = temp[:len(temp) - 5]
  sentence = temp + sentence[6:]
  comments_datasets_to_use[2][index] = sentence
comments_datasets_to_use[2][0]

"<bos> ^ the only one I went to is ikenberry, which is nice and has a vast food selection also with enough vegan/gluten free choices, only problem is cost (during school year, maybe? in summer they didn't change for any specific items) and long lines. are any of the other dining halls better/comparable and do they serve special diet foods? The top reply was: All the dining halls are all you can eat, the A La Carte locations (listed above by /u/palmtree5) are pay per item. No ripoffs, just sometimes meh food. Download the UI Dining app to see the menus for each meal on a daily basis! So how is the only problem cost? It's all the same.... If you're going into a dining hall for a meal, it's all you can eat. There's no pricing of individual items there. If you're going to one of [these]([url link] that's where the items are priced individually. LAR has a lot of vegan options as well. You can look through the menus of all the dining halls [here]([url link] Be sure to change the date to one 

In [ ]:
generations, rge_dict = evaluate(model1, posts_datasets_to_use[2], tokenizer, batch_size=2, input_length=511, output_length=512,
                                 eval_com=comments_datasets_to_use[2])

Creating datasets
Running predictions


1031it [16:30,  1.04it/s]


Scoring predictions
Evaluation complete!


In [ ]:
print(rge_dict)

{'rouge1': {'p': 1.0, 'r': 0.36558166900963535, 'f': 0.5354226368237165}, 'rouge2': {'p': 1.0, 'r': 0.36068220869782963, 'f': 0.5301490772676477}, 'rougeL': {'p': 1.0, 'r': 0.36558166900963535, 'f': 0.5354226368237165}}


In [ ]:
print(generations.predicted_text.values)
print(generations.input_text.values)

[' The instructors are Cross, Gott, Durkin, and Stabler. Thanks! The top reply was: '
 ' Anyone have any recommendations for good art or design classes? Maybe something about product design, or about typography - anything that is different from engineering classes but is still useful and fun. Any class, regardless of college it falls under, is ok with me. I have a good experience in photoshop (some illustrator) if that helps at all The top reply was: '
 ' any guesses or insight on who the speaker will be for the May ceremony? The top reply was: '
 ...
 " I'm living at PAR and have a mid-loft bed. I was thinking of getting a 3 drawer storage unit to put under my bed to store things but I'm not sure if it'll fit? If it doesn't sit, would it be a good idea to get something like that for the inside of my closet to keep things like shoes and purses? Thanks guys!!! The top reply was: "
 " I'm moving out of state, and I only have one car to transport all of my belongings. I can't see fitting 